<a href="https://colab.research.google.com/github/dmburns1729/Class-Files/blob/main/BREAKOUT_Wk18_Lecture02_CodeAlong_UFOs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wk18 Lecture02 CodeAlong: UFOs

In [ ]:
## Update packages
# !pip install scikit-learn==1.1.3
!pip install pandas==1.5.2
# !pip install matplotlib==3.6.2
!pip install seaborn==0.12.1
!pip install statsmodels==0.13.2
!pip install scipy==1.9.3

## Clear Display
from IPython.display import clear_output
clear_output()

In [ ]:
## Confirm package versions
# import sklearn
# print(f"Sklearn version: {sklearn.__version__}")

import pandas as pd
print(f"Pandas version: {pd.__version__}")

import matplotlib as mpl
print(f"Matplotlib version: {mpl.__version__}")

import seaborn as sns
print(f"Seaborn version: {sns.__version__}")

import scipy
print(f"Scipy version: {scipy.__version__}")

## Learning Objectives

- By the end of this CodeAlong, students will be able to:
   - Calculate time series statistics (rolling mean/std/diff/pct_change
   - Perform feature engineering for time series EDA
   - Aggregate time series using date parts to answer stakeholder questions.

    

# 🕹️Part 1) Preparing Irregular-Interval Time Series

### Overview from Last Lecture

- 1) [ ] Convert the dates & times to a single column (if needed).
- 2) [ ] Convert the datetime column  (most likely a string) to a datetime data type.
- 3) [ ] Set the datetime column as the Series/DataFrame index
- 4) [ ] Resample the time series to the desired/correct frequency using the desired/correct aggregation method.
- 5) [ ] Impute null values (if required)


### UFO Sightings

- UFO Sightings: https://www.kaggle.com/datasets/NUFORC/ufo-sightings

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mticks
import seaborn as sns


import missingno as miss
import datetime as dt
import statsmodels.tsa.api as tsa

In [ ]:
url = "https://raw.githubusercontent.com/coding-dojo-data-science/data-viz-wk18-time-series-codealongs/main/Data/ufos-kaggle/scrubbed.csv"
ufo  = pd.read_csv(url, low_memory=False)
ufo

In [ ]:
ufo.info()

>- 1) [x] Convert the dates & times to a single column (if needed).

## Preparing the Datetime Index

### Converting Date Cols to Datetime

In [ ]:
try:
    ufo['datetime'] = pd.to_datetime(ufo["datetime"])
    display(ufo.head())
except Exception as e:
    display(e)

### Handling Errors with pd.to_datetime

- Can use the `errors` argument for pd.to_datetime:
    - "raise" (default): raise an exception when errors happen
    - 'ignore': ignores the errors and returns the original value for that row.
        - NOT RECOMMENDED: the entire column will not be datetime.
    - 'coerce': convert any bad datetime values to null values (NaT - NotATime)

>- **Branch point: we have a choice on how we deal with the bad timestamps.**
    -  Do we coerce them, make then null values, and drop them? Potentially losing a lot of data.
    - Or do we investigate a bit more to see if we can fix the problem without losing data.
    
    
- Let's see how much data we would lose if we chose to coerce the bad values:

In [ ]:
##check nulls before coercing errors
ufo.isna().sum()

In [ ]:
## saving a copy of original datetime column
ufo['datetime-original'] = ufo['datetime'].copy()
ufo

In [ ]:
## Using errors = "coerce"
ufo["datetime-coerce"] = pd.to_datetime(ufo["datetime"], errors='coerce')
ufo["datetime-coerce"]

In [ ]:
## How many null values did we create?
ufo.isna().sum()

In [ ]:
## What % of dates became null?
ufo.isna().sum()/len(ufo)

In [ ]:
## inspecting just the bad rows
bad_rows = ufo[ufo['datetime-coerce'].isna()]
bad_rows

In [ ]:
## let's try to convert the bad rows again
try:
    pd.to_datetime(bad_rows['datetime-original'])
except Exception as e:
    display(e)

> 🤔"`ParserError: hour must be in 0..23: 10/11/2006 24:00 present at position 0`"

In [ ]:
bad_rows['datetime-original']

> Panda's is confused by 24:00. It doesn't know if we mean 0:00 of the NEXT day or if we mean the 11:59 pm (23:59) the same day

In [ ]:
## replace 24:00 with 23:59
ufo['datetime-fixed'] = ufo["datetime-original"].str.replace(" 24:00"," 23:59")
ufo

In [ ]:
## use the fixed-datetime column to make the datetime col
ufo['datetime'] = pd.to_datetime(ufo["datetime-fixed"])
ufo

In [ ]:
ufo.info()

>- 2) [x] Convert the datetime column  (most likely a string) to a datetime data type.

### Setting datetime index

In [ ]:
ufo_ts = ufo.set_index('datetime')
ufo_ts

In [ ]:
dt_cols = [c for c in ufo_ts.columns if 'datetime' in c]
ufo_ts = ufo_ts.drop(columns=dt_cols)
ufo_ts

In [ ]:
# check the index to confirm its datetime
ufo_ts.index

- 3) [x] Set the datetime column as the Series/DataFrame index

## Let's visualize Our Data

In [ ]:
ufo_ts

In [ ]:
ufo_ts.plot()#y='duration (seconds)')

> Hmmmm.... what are we *trying* to visualize?



### What do we really want to know about UFO's?
- Duration of sighting?
- Location of sighting?
- Number of sightings?

## Getting Our Time Series of UFO Sightings

- We want to quantify the number of events that occurred within each interval.

>- Q: How could we do this? (there's several ways)


In [ ]:
## A.1) Add a count column with a value of 1 for each row and then use reasmple().sum()
demo_ts = ufo_ts.copy()
demo_ts['Count'] = 1
demo_ts

In [ ]:
# A.2)
demo_ts.resample('D').sum()#['Count']

In [ ]:
## B) use .size
ufo_ts.resample("D").size()

### Make `ts` from ufo_ts

In [ ]:
ts = ufo_ts.resample('D').size()
ts

In [ ]:
plt.rcParams['figure.figsize'] = [10,5]

In [ ]:
ts.plot();

> Let's keep data from 1950 to present day

In [ ]:
ts = ts.loc["1950":]
ts

In [ ]:
ts.plot()

In [ ]:
## save our new frequency ts and plot
ts_m = ts.resample("M").sum()
ts_m.plot();

# 🕹️Part 2) Aggregating Full Dataset Using Date Parts

## 📝 **Stakeholder Questions to Answer**

**ANSWER TOGETHER:**
- 1) What Month/Year had the most sightings? (and how many sightings were there?)

- 2) Which month of the year has the highest number of reported sightings?
- 3) Is there a seasonal pattern to UFO sightings? If so, how long is the season?

- 4) Which US holiday has the largest number of sightings?
___
**ANSWER SELECTED Q's IN BREAKOUT ROOMS**

- 5) Which year had the highest % increase in sightings compared to previous years? (since 1950)

- 6) What day of the week has the highest reported sightings?

- 7) At what time of day (hour) do most sightings occur?

- 8) Which US state has the most sightings?

- 9) Which country had the largest proportion of sightings for the year 2000?

- 10) Have the types/shapes of UFO's witness changed over time?
    - Tip: use only the 4 most common shapes

### Making `eda_df` for answering questions

In [ ]:
## making eda_df with date as a column instead of index
eda_df = ufo_ts.reset_index()
eda_df

### Feature Engineering: Date Parts

- Datetime objects have:
    - year
    - month
    - month_name()
    - day
    - day_name()
    - hour
    - seconds
    
- Pandas has a `.dt.` accessor to use datetime methods on an entire column at once.

In [ ]:
## feature engineering for dates
eda_df['year'] = eda_df['datetime'].dt.year
eda_df['month'] = eda_df['datetime'].dt.month_name()
eda_df['day of month'] = eda_df['datetime'].dt.day
eda_df['day of week'] = eda_df['datetime'].dt.day_name()
eda_df['hour'] = eda_df['datetime'].dt.hour
eda_df

> Let's add a "weekend" feature that will be True if the day was a Saturday or Sunday.

In [ ]:
## let's add a weekend feature
eda_df['weekend'] = eda_df['day of week'].isin(['Saturday','Sunday'])
eda_df

#### Let's add a column for the decade

In [ ]:
## Calculate decade by subtracting the remainder and div by 10
eda_df['decade'] = eda_df['year'] - eda_df['year']%10
eda_df

In [ ]:
# ## Original year
# year = eda_df['year']
# year

In [ ]:
# ## remainder / by 10 years
# remainder = year%10
# remainder

In [ ]:
# ## Subtract to get decade
# year-remainder

### Making `eda_ts` & `ts`

In [ ]:
## Making eda_ts with dt index
eda_ts = eda_df.set_index('datetime')

## Resampling to Monthly data
ts = eda_ts.resample("M").size().loc['1950':]
ts

## Answering Stakeholder Questions (Together)

### 1) What Month/Year had the most sightings? (and how many sightings were there?)


In [ ]:
date_most_ufos = ts.idxmax()
date_most_ufos

In [ ]:
ts.loc[date_most_ufos]

In [ ]:
ax = ts.plot();
fmt = "%m/%Y"
ax.axvline(date_most_ufos, ls='--',color='k',
           label=f"{date_most_ufos.strftime(fmt)} had {ts.loc[date_most_ufos]}")
ax.legend()

### 2) Which month of the year has the highest number of reported sightings?

In [ ]:
eda_ts['month'].value_counts()


### 3) Is there a seasonal pattern to UFO sightings? If so, how long is the season?

### Seasonality

In [ ]:
import statsmodels.tsa.api as tsa

In [ ]:
# ts_m.loc[date_slice]
ts_check_season = ts_m.loc["2000":]
ts_check_season

In [ ]:
ts_check_season.plot();

In [ ]:
decomp = tsa.seasonal_decompose(ts_check_season)
decomp.plot();

In [ ]:
seasonal = decomp.seasonal
ax = seasonal.plot(figsize=(12,3))
ax.set(ylabel='Change in # of Sightings',
      title='Seasonal Component of Sightings');

#### Using scipy's find_peaks

In [ ]:
from scipy.signal import find_peaks
peaks, props = find_peaks(seasonal, height=seasonal.max())

In [ ]:
peaks

In [ ]:
peak_dates = seasonal.index[peaks]
peak_dates

### 4) Which US holiday has the largest number of sightings?

#### Feature Engineering: Holidays

In [ ]:
# !pip install holidays
import holidays
import datetime as dt
from holidays import country_holidays

In [ ]:
## Create an instance of the US country holidays.
us_holidays = country_holidays('US')
us_holidays

In [ ]:
## create a test holiday
test = "01/01/2015"
test

In [ ]:
## test the api
us_holidays.get(test)

In [ ]:
## Saving both holiday types as columns
eda_df["US Holiday"] = eda_df['datetime'].map(us_holidays.get)
eda_df

In [ ]:
eda_df['US Holiday'].unique()

#### Answer to which holiday has most sightings:

In [ ]:
eda_df['US Holiday'].value_counts()

In [ ]:
sns.countplot(data=eda_df, y='US Holiday');

#### Wait...when did that movie come out?

In [ ]:
release_date= '1997-07-03'

In [ ]:
## Plot the # of sightings over time and annotate the release date
ax = ts.plot()
# rel_date = '1997-05-26'
ax.axvline(release_date, label=f'Indepence Day Released {release_date}',
          ls='--',color='k');
ax.legend();

## 🏓**Breakout Rooms: Answering Stakeholder Questions**

**Choose 1-2 of the remaining questions and work in breakout rooms to answer them:**
- 5) Which year had the highest % increase in sightings compared to previous years?
- 6) What day of the week has the highest reported sightings?
- 7) At what time of day (hour) do most sightings occur?
- 8) Which US state has the most sightings?
- 9) Which country had the largest proportion of sightings for the year 2000?
- 10) Have the types/shapes of UFO's witness changed over time?
    - Tip: use only the 4 most common shapes



### 5) Which year had the highest % increase in sightings compared to previous years? (since 1950)

### 6) What day of the week has the highest reported sightings?

### 6) Which country had the largest proportion of sightings for the year 2000?

### 7) Have the types/shapes of UFO's witness changed over time?

___
# Bonus: Plotly Express

In [ ]:
import plotly.express as px
import plotly.io as pio

### Map Over Time

In [ ]:
eda_df = eda_df.sort_values('decade')
eda_df.columns = eda_df.columns.str.strip()
eda_df['latitude'] = pd.to_numeric(eda_df['latitude'], errors='coerce')
eda_df.head()

In [ ]:
px.scatter_geo(data_frame=eda_df, lat='latitude',lon='longitude', animation_frame="decade",
              template='ggplot2')